In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmnist-typeface-mnist/TMNIST_Data.csv


# Data Science Engineering Methods and Tools

**Assignment- **

In [3]:
df = pd.read_csv("/kaggle/input/tmnist-typeface-mnist/TMNIST_Data.csv")

In [ ]:
df.head()

In [ ]:
df = df.drop(['names'],axis=1)

In [ ]:
df.head()

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# lb = LabelEncoder()
# df['labels'] = lb.fit_transform(df['labels'])

In [ ]:
df.head()

In [ ]:
df['labels'].nunique()

In [ ]:
X = df.drop(['labels'],axis=1)
y = df['labels']


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
# importing the neccesary libraries
import keras 
from keras.models import Model
from keras.layers import Dense,Input,Dropout
from keras.layers import Conv2D,MaxPooling2D, Activation,Flatten, BatchNormalization

#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical


In [ ]:
#now type casting the training and testing data to float
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

#normalize the data
X_train = X_train/255
X_test = X_test/255



In [ ]:
y_train = to_categorical(y_train)

In [ ]:
y_test = to_categorical(y_test)

In [ ]:
print(y_train.shape)

In [ ]:
# now in order to define the channel size in the Convolutional Neural Network we have to reshape X_train and X_test
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)



In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
# creating the neural network
model = Sequential()

# first convolutional layer


model.add(Conv2D(128,(3,3),input_shape = X_train.shape[1:])) # first conv layer just to mention input size
model.add(BatchNormalization())
model.add(Activation("relu")) # activation function to maki it non linear
model.add(MaxPooling2D(pool_size=(2,2)))

# second convolutional layer


model.add(Conv2D(128,(3,3))) # second conv layer 
model.add(BatchNormalization())
model.add(Activation("relu")) # activation function to maki it non linear
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# third convolutional layer

model.add(Conv2D(64,(3,3))) # third conv layer
model.add(BatchNormalization())
model.add(Activation("relu")) # activation function to maki it non linear
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))




# fully connected layer 1
model.add(Flatten())  # converts 2d to 1d into 1 feature vector
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.25))

# fully connected layer 2

model.add(Dense(64))
model.add(Activation("relu"))

# last fully connected layer

model.add(Dense(10)) # last layer should contain 10 neurons as we have to classsify output labels into 10 categories
model.add(Activation("softmax")) # softmax ac fun for classification

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,epochs=5,validation_split=0.3,batch_size = 32)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming 'model' is your Keras model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
!pip install visualkeras


In [ ]:
import visualkeras
from tensorflow.keras import layers
from collections import defaultdict
from PIL import Image

# Creating a color map to customize the colors for different layers
color_map = defaultdict(dict)
color_map[layers.Conv2D]['fill'] = '#00f5d4'
color_map[layers.MaxPooling2D]['fill'] = '#8338ec'
color_map[layers.Dropout]['fill'] = '#03045e'
color_map[layers.Dense]['fill'] = '#fb5607'
color_map[layers.Flatten]['fill'] = '#ffbe0b'

# Generate a layered view of the model with customized colors
image = visualkeras.layered_view(model, legend=True, color_map=color_map)

# Save the image
image_path = 'model_visual.png'
image.save(image_path,dpi=(500, 500))

# Display the image
display(Image.open(image_path))

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot

# Create traces
trace0 = go.Scatter(
    x = list(range(1, len(history.history['accuracy']) + 1)),
    y = history.history['accuracy'],
    mode = 'lines+markers',
    name = 'Train Accuracy'
)
trace1 = go.Scatter(
    x = list(range(1, len(history.history['val_accuracy']) + 1)),
    y = history.history['val_accuracy'],
    mode = 'lines+markers',
    name = 'Test Accuracy'
)

data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Model Accuracy',
              xaxis = dict(title = 'Epoch'),
              yaxis = dict(title = 'Accuracy'),
              )

fig = dict(data=data, layout=layout)
iplot(fig)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
plt.figure(figsize=(20, 20))
num_features = min(layer_activation.shape[-1], 36)  # Limit to 36 feature maps
for i in range(num_features):
    plt.subplot(6, 6, i + 1)
    plt.imshow(layer_activation[0, :, :, i], cmap='viridis')
    plt.axis('off')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        # Assuming the first element in the weights list is the weight tensor
        layer_weights = weights[0]
        plt.figure(figsize=(10, 5))
        sns.histplot(layer_weights.ravel(), kde=True)  # Flatten and plot
        plt.title(f'Weight Distribution in Layer: {layer.name}')
        plt.xlabel('Weights')
        plt.ylabel('Frequency')
        plt.show()
